# Let's Get Relational

Building databases with Python's SQLite3 in 10 (relatively) easy steps

## What is a SQLite database?

#### A group of datasets joined by keys
The simplest way to think about a relational database is as a series of datasets in which a special "key" column in one dataset references a specific row in another dataset, allowing you to join and retrieve related pieces of information on quickly and simply.

#### That is stored locally on your computer
Unlike many other relational database engines, like PostgreSQL or MySQL, which can be storage intensive and difficult to set up locally, SQLite is just one self-contained file.

## Example

#### You have the following datasets:
- Demographics by CSA (combined statistical area) in Baltimore
- List of polling locations in Baltimore
- Turnout by voter district in Baltimore
- 9-1-1 Calls

#### And you want to retreive these subsets of the combined data:
- A list of the polling locations in CSAs with populations that are at least 75% white
- The demographic breakdown of CSAs that overlap with the voting district with the highest turnout
- All of the 911 calls that placed within 1 mile of a polling location on election day

## Why are they useful?

- Replace multiple spreadsheets with a single file
- Merge once, share endlessly
- Query what you need, when you need it

## When do they come in handy?

#### If your analysis...
- Rivals Frankenstein in the number of sources it stitches together
- Calls `merge()` and `groupby()` 50 times before returning the first meaningful result
- Takes longer to load than the family van on summer vacation

#### ...you might want to use a SQLite database

## Okay, okay how do I actually create one?

### Let's start with something simple

Combining several playlists into a single music library with the following information:
- Album
- Artist
- Duration
- Genre

### Step 1: Import pandas and playlist for cleanup

In [2]:
import pandas as pd

#read entire excel file to avoid multiple reads
xlsx = pd.ExcelFile("P4GC_SQLite3_Test.xlsx")

#read each sheet to a different dataframe
df_gritty = pd.read_excel(xlsx, "Gritty Playlist")
df_mix = pd.read_excel(xlsx, "Mix Tape Playlist")
df_alt = pd.read_excel(xlsx, "Alt Playlist")
df_rainy = pd.read_excel(xlsx, "Rainy Day Playlist")

### Step 2: Data munging magic

![magic](https://media.giphy.com/media/GjPf6uoxopCIE/giphy.gif)

In [10]:
#add corresponding playlist name to new playlist column
df_gritty["Playlist"] = "Gritty"
df_mix["Playlist"] = "Mix Tape"
df_alt["Playlist"] = "Alt"
df_rainy["Playlist"] = "Rainy Day"

#combine playlists and reset the index
df_all = pd.concat([df_gritty, df_mix, df_alt, df_rainy], ignore_index=True)

In [11]:
#convert the duration timestamp to a duration delta
df_all["Duration_Delta"] = df_all["Duration"].apply(lambda x: pd.Timedelta(days=0,
                                                                           minutes=x.minute,
                                                                           seconds=x.second))
#extract the total duration in seconds from the duration delta
df_all["Duration_Seconds"] = df_all["Duration_Delta"].dt.seconds

#export the records to a list of dicts
records_raw = df_all.to_dict("records")

In [12]:
#inserts a record for each artist on a song
artist_records = [{"Artist": y, 
                   "Album": x["Album"],
                   "Song": x["Song"], 
                   "Playlist": x["Playlist"],
                   "Duration_Delta": x["Duration_Delta"],
                   "Duration_Seconds": x["Duration_Seconds"]}
                  for x in records_raw for y in x["Artist"]]

#converts the list of records back into dataframes for further manipulation
df_artist_raw = pd.DataFrame(artist_records)

In [13]:
#inserts a record for each genre on a song
genre_records = [{"Genre": y, 
                  "Album": x["Album"],
                  "Song": x["Song"], 
                  "Playlist": x["Playlist"],
                  "Duration_Delta": x["Duration_Delta"],
                  "Duration_Seconds": x["Duration_Seconds"]}
                 for x in records_raw for y in x["Genre"]]

#converts the list of records back into dataframes for further manipulation
df_genre_raw = pd.DataFrame(genre_records)

In [14]:
#pulls unique album names and aggregates song count and duration
df_album = (df_all.groupby("Album", as_index=False)
                  .agg({"Song": "count",
                        "Duration_Seconds": "sum",
                        "Duration_Delta": "sum"}))

#drops songs that appear in multiple playlists
df_song = df_all.drop_duplicates(["Album","Song"])

In [15]:
#pulls unique album names and aggregates song count and duration 
df_playlist = (df_all.groupby("Playlist", as_index=False)
                     .agg({"Song": "count",
                           "Duration_Seconds": "sum",
                           "Duration_Delta": "sum"}))

#pulls unique genres and aggregates song count and duration
df_genre = (df_genre_raw.drop_duplicates(["Album","Song"])
                        .groupby("Genre", as_index=False)
                        .agg({"Song": "count",
                              "Duration_Seconds": "sum",
                              "Duration_Delta": "sum"}))

#pulls unique artists and aggregates song count and duration
df_artist = (df_artist_raw.drop_duplicates(["Artist","Song"])
                          .groupby("Artist", as_index=False)
                          .agg({"Song": "count",
                                "Duration_Seconds": "sum",
                                "Duration_Delta": "sum"}))

### Step 3: Install SQLite and sqlite3 and create your database

1. Install sqlite and GUI interface for the database by downloading this app: http://sqlitebrowser.org/

1. Install sqlite3 package to manage sqlite databases with python by executing `! pip install sqlite3`

1. Create your database file by executing `! touch Album.db`

### Step 4: Import the library and connect to your database

In [23]:
import sqlite3
conn = sqlite3.connect("AlbumDB.db")
c = conn.cursor()

### Step 5: Create and insert the table without foreign keys

#### Start with the albums...

In [16]:
#convert df to list of tuples and reformat each value for entry to database
album_records = [(x["Album"],
                  str(x["Duration_Delta"]),
                  int(x["Duration_Seconds"]), 
                  int(x["Song"]))
                 for x in df_album.to_dict("records")]

In [18]:
#creates album table
c.execute('DROP TABLE IF EXISTS Album')
c.execute('''CREATE TABLE Album (
                Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                Name TEXT,
                Duration_Text TEXT,
                Duration_Seconds INTEGER,
                Song_Count INTEGER
             )''')
#inserts values from album_record list
c.executemany('''INSERT INTO Album (Name, Duration_Text, Duration_Seconds, Song_Count)
                 VALUES (?,?,?,?)''',
              album_records)
#commits changes to the database
conn.commit()

In [ ]:
#convert df to list of tuples and reformat each value for entry to database
playlist_records = [(x["Playlist"],
                     str(x["Duration_Delta"]),
                     int(x["Duration_Seconds"]), 
                     int(x["Song"]))
                    for x in df_playlist.to_dict("records")]

In [ ]:
#creates playlist table
c.execute('DROP TABLE IF EXISTS Playlist')
c.execute('''CREATE TABLE Playlist (
                Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                Name TEXT,
                Duration_Text TEXT,
                Duration_Seconds INTEGER,
                Song_Count INTEGER
             )''')
#inserts values from playlist_record list
c.executemany('''INSERT INTO Playlist (Name, Duration_Text, Duration_Seconds, Song_Count)
                 VALUES (?,?,?,?)''',
              playlist_records)
#commits changes to the database
conn.commit()

In [ ]:
#convert df to list of tuples and reformat each value for entry to database
artist_records = [(x["Artist"],
                   str(x["Duration_Delta"]),
                   int(x["Duration_Seconds"]), 
                   int(x["Song"]))
                  for x in df_artist.to_dict("records")]

In [ ]:
#creates Artist table
c.execute('DROP TABLE IF EXISTS Artist')
c.execute('''CREATE TABLE Artist (
                Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                Name TEXT,
                Duration_Text TEXT,
                Duration_Seconds INTEGER,
                Song_Count INTEGER
             )''')
#inserts values from artist_records list
c.executemany('''INSERT INTO Artist (Name, Duration_Text, Duration_Seconds, Song_Count)
                 VALUES (?,?,?,?)''',
              artist_records)
#commits changes to the database
conn.commit()

In [ ]:
#convert df to list of tuples and reformat each value for entry to database
genre_records = [(x["Genre"],
                  str(x["Duration_Delta"]),
                  int(x["Duration_Seconds"]), 
                  int(x["Song"]))
                 for x in df_genre.to_dict("records")]

In [ ]:
#creates genre table
c.execute('DROP TABLE IF EXISTS Genre')
c.execute('''CREATE TABLE Genre (
                Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                Name TEXT,
                Duration_Text TEXT,
                Duration_Seconds INTEGER,
                Song_Count INTEGER
             )''')
#inserts values from artist_records list
c.executemany('''INSERT INTO Genre (Name, Duration_Text, Duration_Seconds, Song_Count)
                 VALUES (?,?,?,?)''',
              genre_records)
#commits changes to the database
conn.commit()

### Step 6: rinse and repeat for playlists, artists, and genres

### Step 7: Query records, merge, and repeat the process for tables with foreign keys

#### Start with the songs...

In [19]:
#query and retrieve name and id field from album table
c.execute('SELECT Id, Name FROM Album')
album_ids = c.fetchall()

In [21]:
#convert to dataframe then merge with df_song
df_album_id = pd.DataFrame(album_ids, columns=["Id", "Album"])

df_song_join = df_song.merge(df_album_id, on="Album")
df_song_join.head(3)

,Song,Artist,Album,Duration,Genre,Playlist,Duration_Delta,Duration_Seconds,Id
0,S.O.B.,Nathaniel Rateliff & The Night Sweats,Nathaniel Rateliff & The Night Sweats,00:04:08,Rock;Soul;Blues,Gritty,00:04:08,248,11
1,Howling At Nothing,Nathaniel Rateliff & The Night Sweats,Nathaniel Rateliff & The Night Sweats,00:03:10,Soul;Blues;Rock,Gritty,00:03:10,190,11
2,Lengths,The Black Keys,Rubber Factory,00:04:52,Blues;Rock;Folk,Gritty,00:04:52,292,15


In [ ]:
#convert df to list of tuples and reformat each value for entry to database
song_records = [(x["Song"],
                 str(x["Duration_Delta"]),
                 int(x["Duration_Seconds"]), 
                 int(x["Id"]))
                for x in df_song_join.to_dict("records")]

In [ ]:
#creates song table
c.execute('DROP TABLE IF EXISTS Song')
c.execute('''CREATE TABLE Song (
                Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                Name TEXT,
                Duration_Text TEXT,
                Duration_Seconds INTEGER,
                Album_Id INTEGER,
                FOREIGN KEY(Album_Id) REFERENCES Album (Id)
             )''')
#inserts values from artist_records list
c.executemany('''INSERT INTO Song (Name, Duration_Text, Duration_Seconds, Album_Id)
                 VALUES (?,?,?,?)''',
              song_records)
#commits changes to the database
conn.commit()

### Step 8: Rinse and repeat to join playlists, songs, genres, and artists 

In [24]:
#query and retrieve name and id field from song table
c.execute('SELECT Id, Name FROM Song')
song_ids = c.fetchall()

#query and retrieve name and id fields from playlist table
c.execute('SELECT Id, Name FROM Playlist')
playlist_ids = c.fetchall()

In [ ]:
#convert results to dataframes
df_song_id = pd.DataFrame(song_ids, columns=["Song_Id", "Song"])
df_playlist_id = pd.DataFrame(playlist_ids, columns=["Playlist_Id", "Playlist"])

In [ ]:
df_playlist_song1 = df_all.merge(df_song_id, on="Song")
df_playlist_song2 = df_playlist_song1.merge(df_playlist_id, on="Playlist")

In [ ]:
#creates playlist song table
c.execute('DROP TABLE IF EXISTS Playlist_Song')
c.execute('''CREATE TABLE Playlist_Song (
             Song_Id INTEGER,
             Playlist_Id INTEGER,
             PRIMARY KEY(Song_Id, Playlist_Id),
             FOREIGN KEY(Song_Id) REFERENCES Song (Id),
             FOREIGN KEY(Playlist_Id) REFERENCES Playlist (Id))''')

c.executemany('INSERT INTO Playlist_Song (Song_ID, Playlist_ID) VALUES (?, ?)',
              playlist_song_records)
conn.commit()

### Step 9:  Query a subset of records you just inserted

#### Retrieve all of the albums with at least one song on the Mix Tape playlist

In [28]:
c.execute("""SELECT Album.Name AS 'Album', Song.Name AS 'Song', Album.Duration_Text as 'Length', 
             Playlist.Name as 'Playlist' FROM Album, Song, Playlist, Playlist_Song
             WHERE Song.Album_Id = Album.Id 
             AND Song.Id = Playlist_Song.Song_Id
             AND Playlist_Song.Playlist_Id = Playlist.Id
             AND Playlist.Name = 'Mix Tape'""")
new_query = c.fetchall()

#### Convert to a dataframe and view your results

In [30]:
df_query = pd.DataFrame(new_query)
df_query.columns = ['Album', 'Song', 'Album Length', 'Playlist']
df_query.head()

,Album,Song,Album Length,Playlist
0,Skeptic Goodbyes,Who Knew,0 days 00:09:00,Mix Tape
1,Ben Kweller,Thirteen,0 days 00:12:40,Mix Tape
2,White Blood Cells,We're Going To Be Friends,0 days 00:02:22,Mix Tape
3,"Peaceful, The World Lays Me Down",5 Years Time,0 days 00:05:40,Mix Tape
4,"Peaceful, The World Lays Me Down",2 Atoms In A Molecule,0 days 00:05:40,Mix Tape


### Step 10: Share your newly polished database and pat yourself on the back

![self-five](https://i1.wp.com/gifrific.com/wp-content/uploads/2012/04/Tina-Fey-giving-herself-high-five.gif?ssl=1)